In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm

# Define your CNN-BiLSTM model
class CNNBiLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes, kernel_size, dropout_prob):
        super(CNNBiLSTM, self).__init__()

        # CNN layer
        self.cnn = nn.Conv1d(in_channels=embedding_dim, out_channels=hidden_dim, kernel_size=kernel_size)

        # BiLSTM layer
        self.bilstm = nn.LSTM(input_size=hidden_dim, hidden_size=hidden_dim, bidirectional=True, batch_first=True)

        # Fully connected layer
        self.fc = nn.Linear(hidden_dim * 2, num_classes)

        # Dropout for regularization
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        # Input shape: (batch_size, sequence_length, embedding_dim)
        x = x.permute(0, 2, 1)  # Reshape for CNN input: (batch_size, embedding_dim, sequence_length)

        # CNN
        x = self.cnn(x)

        # BiLSTM
        x, _ = self.bilstm(x)

        # Global max pooling
        x = torch.max(x, dim=1).values

        # Fully connected layer
        x = self.fc(x)

        return x

# Set hyperparameters
vocab_size = 10000  # Adjust based on your dataset
embedding_dim = 100
hidden_dim = 64
num_classes = 2
kernel_size = 3
dropout_prob = 0.5

# Instantiate the model
model = CNNBiLSTM(vocab_size, embedding_dim, hidden_dim, num_classes, kernel_size, dropout_prob)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Set up your optimizer and loss function (adjust as needed)
optimizer = optim.Adam(model.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        inputs, labels = batch['input_ids'].to(device), batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(validation_dataloader, desc="Validation"):
        inputs, labels = batch['input_ids'].to(device), batch['labels'].to(device)
        outputs = model(inputs)
        predictions = torch.argmax(outputs, dim=1)

        all_preds.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")
